In [ ]:
import pandas as pd
from fpdf import FPDF

def frage_leistungsphase(name, prozentsatz, grundhonorar, daten_dict):
    while True:
        try:
            eingabe = input(f"{name} j/n: ").lower()
            if eingabe not in ("j", "n"):
                raise ValueError("Bitte nur mit 'j' oder 'n' antworten.")
            if eingabe == "j":
                daten_dict[name] = prozentsatz * 100
                daten_dict[f"Betrag {name}"] = round(grundhonorar * prozentsatz, 2)
            else:
                daten_dict[name] = "nein"
                daten_dict[f"Betrag {name}"] = 0.00
            break
        except ValueError as e:
            print(e)

def fragebogen():
    leistungsphasen = {
        "Grundlagenermittlung": 0.02,
        "Vorplanung": 0.09,
        "Entwurfsplanung": 0.17,
        "Genehmigungsplanung": 0.02,
        "Ausführungsplanung": 0.22,
        "Vorbereitung der Vergabe": 0.07,
        "Mitwirkung bei der Vergabe": 0.05
    }

    print("=== Fragebogen zur Angebotserstellung ===\n")
    daten_projekt, daten_bauherr, daten_honorar, daten_leistungen, daten = {}, {}, {}, {}, {}

    print("=== Information über das Projekt ===\n")
    daten_projekt["Firma/Name"] = input("Firma / Name: ")
    daten_projekt["Ansprechpartner"] = input("Ansprechpartner: ")
    daten_projekt["Adresse"] = input("Adresse des Projekts: ")
    daten_projekt["Telefonnummer"] = input("Telefonnummer: ")
    daten_projekt["E-Mail"] = input("E-Mail-Adresse: ")
    daten_projekt["Projektbezeichnung"] = input("Projektbezeichnung / Referenz: ")
    daten_projekt["Projekt-Nr"] = input("Projekt-Nummer: ")

    print("=== Information über den Bauherr ===\n")
    daten_bauherr["Bauherr"] = input("Name des Bauherr: ")
    daten_bauherr["Adresse Bauherr"] = input("Anschrift Bauherr: ")
    daten_bauherr["Kontakt"] = input("Kontaktdaten: ")

    daten_honorar["Kosten"] = float(input("Gesamtkosten (anrechenbar): "))
    while True:
        try:
            hz = int(input("Honorarzone gem. HOAI (1–5): "))
            if hz < 1 or hz > 5:
                raise ValueError("Die Honorarzone muss zwischen 1 und 5 liegen.")
            daten_honorar["Honorarzone"] = hz
            break
        except ValueError as e:
            print(e)

    honorar_tafel = {1: 10.8, 2: 13.4, 3: 15.3, 4: 17.3, 5: 19.0}
    honorar_prozent = honorar_tafel[hz]
    grundhonorar = daten_honorar["Kosten"] * honorar_prozent / 100
    daten_honorar["Honorarsatz"] = f"{honorar_prozent} %"
    daten_honorar["Grundhonorar"] = round(grundhonorar, 2)

    print("=== Projektleistungen / Teilhonorar ===\n")
    for lp, proz in leistungsphasen.items():
        frage_leistungsphase(lp, proz, grundhonorar, daten_leistungen)

    daten_leistungen["Gesamtsumme"] = sum(v for k, v in daten_leistungen.items() if k.startswith("Betrag "))
    daten_leistungen["Gesamtsumme proz."] = round(daten_leistungen["Gesamtsumme"] / grundhonorar * 100, 2)
    nebenkosten_prozent = 0.04
    daten["Nebenkosten"] = round(daten_leistungen["Gesamtsumme"] * nebenkosten_prozent, 2)
    daten["Zwischensumme"] = round(daten_leistungen["Gesamtsumme"] + daten["Nebenkosten"], 2)
    daten["MwSt 19 %"] = round(daten["Zwischensumme"] * 0.19, 2)
    daten["Bruttohonorar"] = round(daten["Zwischensumme"] + daten["MwSt 19 %"], 2)

    class FragebogenPDF(FPDF):
        def header(self):
            # Width of logo in mm
            logo_width = 50

            # x = page width - right margin - logo width
            x_pos = self.w - self.r_margin - logo_width

            self.image("logo.png", x=x_pos, y=10, w=logo_width)

            # "Title" kind of
            self.set_font("Arial", "B", 12)
            self.cell(0, 6, f"Anglage zum Angebot {daten_projekt["Projektbezeichnung"]}", ln=True, align="L")
            self.cell(0, 6, "Honorarabrechnung", ln=True, align="L")
            
            # horizontal line
            self.line(self.l_margin, self.get_y(), self.w - self.r_margin, self.get_y())
            self.ln(2)
    
            self.set_font("Arial", "", 12)
            self.cell(60, 6, f"Projekt: ", align="L")
            self.cell(70, 6, f"{daten_projekt['Projekt-Nr']}", ln=True, align="C")
            self.cell(60, 6, f"Projekt Kurzbezeichnung: ", align="L")
            self.cell(70, 6, f"{daten_projekt['Projektbezeichnung']}", ln=True, align="C")
            self.ln(2)
            self.cell(60, 6, f"Auftraggeber: ", align="L")
            self.cell(70, 6, f"{daten_projekt["Firma/Name"]}", ln=True, align="C")
            self.cell(0, 6, f"{daten_projekt['Adresse']}", ln=True, align="C")
            self.cell(0, 6, "", ln=True)
            self.set_font("Arial", "B", 13)
            self.cell(0, 10, "Honorar-Angebot gem. HOAI", ln=True, align="C")
            self.ln(5)

        def honorar_angebot(self, daten_honorar):
            honorar_klassen = ["I","II","III","IV","V"]
            self.set_font("Arial", "B", 12)
            self.cell(0, 6, "Honorar Angebot", ln=True, align="L")
            self.set_font("Arial", "", 12)
            if daten_honorar["Honorarzone"] == 1:
                self.cell(0, 6, f"Honorarzone gem. § 56 (2)-(3) HOAI: {honorar_klassen[0]}", ln=True, align="C")
            elif daten_honorar["Honorarzone"] == 2:
                self.cell(0, 6, f"Honorarzone gem. § 56 (2)-(3) HOAI: {honorar_klassen[1]}", ln=True, align="C")
            elif daten_honorar["Honorarzone"] == 3:
                self.cell(0, 6, f"Honorarzone gem. § 56 (2)-(3) HOAI: {honorar_klassen[2]}", ln=True, align="C")
            elif daten_honorar["Honorarzone"] == 4:
                self.cell(0, 6, f"Honorarzone gem. § 56 (2)-(3) HOAI: {honorar_klassen[3]}", ln=True, align="C")
            else:
                self.cell(0, 6, f"Honorarzone gem. § 56 (2)-(3) HOAI: {honorar_klassen[4]}", ln=True, align="C")
        
        def kosten_schaetzung(self, daten_honorar):
            self.set_font("Arial", "B", 12)
            self.cell(0, 6, "Kostenschätzung", ln=True, align="L")
            self.set_font("Arial", "", 12)
            self.cell(60, 6, f"Kosten: ", align="L")
            self.cell(70, 6, f"{daten_honorar["Kosten"] } EUR", ln=True, align="C")
            
            self.cell(60, 6, f"Grundhonorar (100%): ", align="L")
            self.cell(70, 6, f"{daten_honorar["Grundhonorar"]} EUR", ln=True, align="C")
        
        def tabelle_leistungsphasen_detailliert(self, daten_leistungen):
            self.set_font("Arial", "B", 11)
            self.cell(10, 8, "Nr.", border=1, align="C")
            self.cell(80, 8, "Leistungsphase", border=1, align="C")
            self.cell(25, 8, "% HOAI", border=1, align="C")
            self.cell(30, 8, "% vereinb.", border=1, align="C")
            self.cell(45, 8, "Betrag [EUR]", border=1, ln=True, align="C")

            self.set_font("Arial", "", 11)
            nummer = 1
            for key in daten_leistungen:
                if key.startswith("Betrag ") and daten_leistungen[key] > 0:
                    lp = key.replace("Betrag ", "")
                    proz = daten_leistungen.get(lp, "")
                    betrag = f"{daten_leistungen[key]:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
                    self.cell(10, 8, str(nummer), border=1, align="C")
                    self.cell(80, 8, lp, border=1)
                    self.cell(25, 8, f"{proz:.2f} %", border=1, align="C")
                    self.cell(30, 8, f"{proz:.2f} %", border=1, align="C")
                    self.cell(45, 8, f"{betrag} EUR", border=1, ln=True, align="R")
                    nummer += 1

            self.set_font("Arial", "B", 11)
            gesamt = daten_leistungen.get("Gesamtsumme", 0.0)
            gesamt_str = f"{gesamt:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
            self.cell(145, 8, "Gesamtsumme", border=1, align="R")
            self.cell(45, 8, f"{gesamt_str} EUR", border=1, ln=True, align="R")
            self.ln(4)

        def zusammenfassung(self, daten):
            self.set_font("Arial", "", 11)
            for key, value in daten.items():
                betrag = f"{value:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
                self.cell(100, 8, f"{key}", align="L")
                self.cell(0, 8, f"{betrag} EUR", ln=True, align="R")

    pdf = FragebogenPDF()
    pdf.add_page()
    pdf.honorar_angebot(daten_honorar)
    pdf.kosten_schaetzung(daten_honorar)
    pdf.tabelle_leistungsphasen_detailliert(daten_leistungen)
    pdf.zusammenfassung(daten)

    pdf_name = f"Fragebogen_Angebotserstellung_{daten_projekt['Firma/Name'].replace(' ', '_')}.pdf"
    pdf.output(pdf_name)
    print(f"PDF-Datei gespeichert als: {pdf_name}")

if __name__ == "__main__":
    fragebogen()


=== Fragebogen zur Angebotserstellung ===

=== Information über das Projekt ===



Firma / Name:  Chartite
Ansprechpartner:  
Adresse des Projekts:  
Telefonnummer:  
